https://www.psycopg.org/docs/usage.html#the-problem-with-the-query-parameters

https://www.psycopg.org/docs/sql.html
psycopg2.sql – SQL string composition¶
New in version 2.7.

In [2]:
import psycopg2
import os, getpass
from psycopg2.sql import SQL, Identifier


In [3]:
def create_server_connection(database, user):
    conn = None
    try:
        conn = psycopg2.connect(
            database=database,
            user=user,
        )
        print(f"Connected to Database {database} successfully")
        
    except Exception as e:
        print(e)

    return conn

In [4]:
def create_or_drop_database(conn, query):
    autocommit = conn.autocommit #preserve the value of autocommit
    conn.autocommit = True #MUST BE set to True since Postgresql cannot create a database in a transaction
    with conn.cursor() as cur:
        try:
            cur.execute(query)
            print(f"Query: {query} ran successfully.")
        except Exception as e:
            print(e)
        finally:
            #be a good citizen and put stuff back the way you found it
            conn.autocommit = autocommit 

In [5]:
#version 2
def execute_query(conn, query):
    autocommit = conn.autocommit #preserve the value of autocommit
    conn.autocommit = True #MUST BE set to True since Postgresql cannot create a database in a transaction
    cur = conn.cursor()
    try:
        cur.execute(query)
        print(f"Query: {query}ran successfully.")
    except Exception as e:
        print(e)
    finally:
        #be a good citizen and put stuff back the way you found it
        conn.autocommit = autocommit 
    return cur

In [6]:
db, user = "jupyterdb", getpass.getuser()
conn = create_server_connection(db,user)

Connected to Database jupyterdb successfully


In [7]:
conn.autocommit = True

In [8]:
# let's create an authors table in our database

In [9]:
query = """
DROP TABLE IF EXISTS author;

CREATE TABLE author(
name text,
email text
);
"""
cur = execute_query(conn, query)
cur.close()

Query: 
DROP TABLE IF EXISTS author;

CREATE TABLE author(
name text,
email text
);
ran successfully.


In [18]:
query = "INSERT INTO author (name) VALUES (%s);"
data = "not-me",  # gotcha: the data argument which is substituted into %s *MUST* BE AN ITERABLE. Tuple used here!
cur = conn.cursor()
cur.execute(query, data) #passing two arguments to cur.execute()!
cur.close()

NOTE: THE ONLY FORMAT STRING IS %s 
    
IT WORKS FOR ALL DATA TYPES!!!!!!!!!!!!!!!!!!!

In [19]:
query = "SELECT * FROM author;"
cur = conn.cursor()
cur.execute(query) #passing one argument to cur.execute()!
print(cur.fetchall())
cur.close()

[('me', None), ('John Donne', 'death@benotproud.com'), ('Octavia Butler', 'kindred@thefutureisnow.com'), ('Alfred Neuman', 'whatmeworry@worryfree.com'), ('not-me', None)]


In [14]:
query = "INSERT INTO author (name, email) VALUES (%s, %s);"
data = ("John Donne", "death@benotproud.com") #any iterable is fine. using tuple here
cur = conn.cursor()
cur.execute(query, data) #passing two arguments to cur.execute()!
cur.close()

In [15]:
query = "SELECT * FROM author;"
cur = conn.cursor()
cur.execute(query) #passing one argument to cur.execute()!
print(cur.fetchall())
cur.close()

[('me', None), ('John Donne', 'death@benotproud.com')]


In [16]:
query = "INSERT INTO author (name, email) VALUES (%s, %s), (%s,%s);"
data = ["Octavia Butler", "kindred@thefutureisnow.com", "Alfred Neuman", "whatmeworry@worryfree.com"] #any iterable is fine. using list here
cur = conn.cursor()
cur.execute(query, data) #passing two arguments to cur.execute()!
cur.close()

In [17]:
query = "SELECT * FROM author;"
cur = conn.cursor()
cur.execute(query)
print(cur.fetchall())
cur.close()

[('me', None), ('John Donne', 'death@benotproud.com'), ('Octavia Butler', 'kindred@thefutureisnow.com'), ('Alfred Neuman', 'whatmeworry@worryfree.com')]


In [ ]:
#now let's change our execute_query to use data in the second argument of cur.execute()

In [20]:
#version 5
def execute_query(conn, *query):
    try:
        autocommit = conn.autocommit #preserve the value of autocommit
        conn.autocommit = True
        cur = conn.cursor()
        
        if len(query) == 2:
            cur.execute(*query)
            print(f"Query: {query[0]} with data:{query[1]} ran successfully.")
        else:
            cur.execute(query[0])
            print(f"Query: {query[0]} ran successfully.")
    
    except Exception as e:
        print(e)
    finally:
        #put stuff back the way you found it
        conn.autocommit = autocommit 
    return cur

In [22]:
query = "INSERT INTO author (name, email) VALUES (%s, %s);"
data = "Mark Haddon", "curious@dog.com"
cur = execute_query(conn, query, data)
cur.close()

Query: INSERT INTO author (name, email) VALUES (%s, %s); with data:('Mark Haddon', 'curious@dog.com') ran successfully.


In [23]:
query = "SELECT * FROM author;"
cur = conn.cursor()
cur.execute(query)
print(cur.fetchall())
cur.close()

[('me', None), ('John Donne', 'death@benotproud.com'), ('Octavia Butler', 'kindred@thefutureisnow.com'), ('Alfred Neuman', 'whatmeworry@worryfree.com'), ('not-me', None), ('Mark Haddon', 'curious@dog.com')]


Let's dive deeper into string composition with the psycopg2.sql module

psycopg2.sql – SQL string composition

The module contains objects and functions useful to generate SQL dynamically, in a convenient and safe way. SQL identifiers (e.g. names of tables and fields) cannot be passed to the execute() method like query arguments:

In [24]:
# This will not work because you cannot interpolate table and field names like other data
cur = conn.cursor()

query = "INSERT INTO %s VALUES (%s, %s)"
data = ['author', 'bret easton ellis', "psycho@americanpsycho.com"] #tries to insert table name 'author' into the data. #WRONG
cur.execute(query, data)

cur.close()

SyntaxError: syntax error at or near "'author'"
LINE 1: INSERT INTO 'author' VALUES ('bret easton ellis', 'psycho@am...
                    ^


In [25]:
# This is the correct way
# we need to use sql.SQL and sql.Identifier insert the table name
# the data stays the same as previous insertions we've done

from psycopg2 import sql
cur = conn.cursor()

query = sql.SQL("INSERT INTO {} VALUES (%s, %s)").format(sql.Identifier('author'))
data = ['bret easton ellis', "psychokiller@americanpsycho.com"]
cur.execute(query, data)

cur.close()

In [26]:
# IT WORKED!!!!
query = "SELECT * FROM author;"
cur = conn.cursor()
cur.execute(query)
print(cur.fetchall())
cur.close()

[('me', None), ('John Donne', 'death@benotproud.com'), ('Octavia Butler', 'kindred@thefutureisnow.com'), ('Alfred Neuman', 'whatmeworry@worryfree.com'), ('not-me', None), ('Mark Haddon', 'curious@dog.com'), ('bret easton ellis', 'psycho@americanpsycho.com')]


In [ ]:
#so, let's dissect the insertion query string a bit

1) use the sql.SQL() function, just like you'd use str.format()
2) inside the argument of format, pass the table name inside sql.Identifier
3) pass the data in the second argument of cur.execute as normal

query = sql.SQL("INSERT INTO {} VALUES (%s, %s)").format(sql.Identifier('author'))
data = ['bret easton ellis', "psychokiller@americanpsycho.com"]


In [27]:
sql.Identifier('author')

Identifier('author')

In [30]:
query = sql.SQL("INSERT INTO {} VALUES (%s, %s)").format(sql.Identifier('author'))
query

Composed([SQL('INSERT INTO '), Identifier('author'), SQL(' VALUES (%s, %s)')])

Notice that our query is now a Composed object that has all the pieces we need to run the query, and we are safe from potential sql injection!

In [31]:
dir(sql) 
'''# there are 
'Composable', 
'Composed', 
'DEFAULT',
'Identifier', 
'Literal',
'NULL', 
'Placeholder', and 
'SQL' objects in the sql module
'''

['Composable',
 'Composed',
 'DEFAULT',
 'Identifier',
 'Literal',
 'NULL',
 'PY3',
 'Placeholder',
 'SQL',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_formatter',
 'ext',
 'string',
 'string_types']

In [ ]:
db, user = "jupyterdb", getpass.getuser()
conn = create_server_connection(db,user)

In [ ]:
query = """
DROP TABLE IF EXISTS author;

CREATE TABLE author(
name text,
email text
);
"""
cur = execute_query(conn, query)
cur.close()

In [ ]:
query, data = sql.SQL("INSERT INTO {} VALUES (%s, %s)").format(sql.Identifier('author')), ["edgar allan poe","raven1845@gmail.com" ]
cur = execute_query(conn, query, data)
cur.close()

In [ ]:
query = "SELECT * FROM author;"
cur = execute_query(conn, query)
cur.fetchall()
cur.close()

In [ ]:
Values containing backslashes and LIKE
Unlike in Python, the backslash (\) is not used as an escape character except in patterns used with LIKE and ILIKE where they are needed to escape the % and _ characters.

This can lead to confusing situations:

>>> path = r'C:\Users\Bobby.Tables'
>>> cur.execute('INSERT INTO mytable(path) VALUES (%s)', (path,))
>>> cur.execute('SELECT * FROM mytable WHERE path LIKE %s', (path,))
>>> cur.fetchall()
[]
The solution is to specify an ESCAPE character of '' (empty string) in your LIKE query:

>>> cur.execute("SELECT * FROM mytable WHERE path LIKE %s ESCAPE ''", (path,))